In [ ]:
# Installs
# You will need at a minimum the following packages. Feel free to install
# additional ones as needed
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 22.0 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

In [ ]:
import random

# Read the already formatted QA dataset
qa_pairs = []

# Read the file where the data is already in the desired format
with open('formatted_qa_pairs.txt', 'r', encoding='utf-8') as f:
    qa_pairs = f.readlines()

# Shuffle the data to avoid any order bias
random.shuffle(qa_pairs)

# Split into 80% train and 20% test
split_index = int(0.8 * len(qa_pairs))
qa_pairs_train = qa_pairs[:split_index]  # 80% for training
qa_pairs_val = qa_pairs[split_index:]  # 20% for testing

# Output the first 5 QA pairs of the train and test set
print("Train dataset (first 5 examples):")
for qa in qa_pairs_train[:5]:
    print(qa)
    print()

print("Test dataset (first 5 examples):")
for qa in qa_pairs_val[:5]:
    print(qa)
    print()

Train dataset (first 5 examples):
<s>[INST]@AI_Supervised_Unsupervised. If we represent clustering as a mapping from x to y, what does y represent?[/INST]Categories or clusters.</s>


<s>[INST]@AI_Supervised_Unsupervised.  Is the concept of "clumps" related to the concept of "density" in data analysis?<binary data, 1 bytes><binary data, 1 bytes><binary data, 1 bytes>[/INST>Yes, "clumps" imply higher density of data points.</s>

<s>[INS]@AI.<question>Q: What is a neural network in supervised learning?</question>[/INS]<answer>A: A neural network is a model composed of layers of interconnected nodes (neurons) that can learn complex patterns in data.</answer></s>


<s>[INST]@AI_Supervised_Unsupervised.What kind of insight can a biologist gain from clustering?.[/INST]Identifying underlying similarities between seemingly dissimilar organisms.</s>


<s>[INST]@AI_Supervised_Unsupervised.What is implied about the nature of the "underlying similarity" between bats and whales?.[/INST]It is a char

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `zxjese577proj` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `zxje

In [ ]:
# Load the model -- Skeleton
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True
)
# unsloth/
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

# 加载Mistral-7B tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):

  tokenizer.padding_side = "right"
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.add_eos_token = True
  tokenizer.bos_token, tokenizer.eos_token

  return tokenizer(
      examples,
      padding=True,
      truncation=True,
      max_length=1024
  )

train_encodings = [preprocess_function(text) for text in qa_pairs_train]
val_encodings = [preprocess_function(text) for text in qa_pairs_val]

dataset_train = {
    'input_ids': [encoding['input_ids'] for encoding in train_encodings],
    'attention_mask': [encoding['attention_mask'] for encoding in train_encodings]
}

dataset_test = {
    'input_ids': [encoding['input_ids'] for encoding in val_encodings],
    'attention_mask': [encoding['attention_mask'] for encoding in val_encodings]
}

dataset_train = Dataset.from_dict(dataset_train)
dataset_test = Dataset.from_dict(dataset_test)

dataset = DatasetDict({"train": dataset_train, "test": dataset_test})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 253
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 64
    })
})


In [ ]:
print(dataset_train['input_ids'][0])
print(dataset_train['attention_mask'][0])

[1, 1, 28792, 16289, 28793, 28818, 11741, 28730, 15503, 23313, 28730, 1733, 8554, 23313, 28723, 1047, 478, 2904, 533, 469, 2131, 390, 264, 13152, 477, 1318, 298, 337, 28725, 767, 1235, 337, 2904, 28804, 28792, 28748, 16289, 28793, 28743, 14922, 442, 20501, 28723, 2, 13, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,  #8 or 16
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, peft_config)

In [ ]:
# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=50,
    logging_dir="./logs",
    logging_steps=5,
    save_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    max_grad_norm=1.0,
    fp16=True,
    gradient_accumulation_steps=2,
)

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: 

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

Epoch,Training Loss,Validation Loss
1,5.468700,No log
2,5.207500,No log
3,4.720900,No log
4,4.593900,No log
5,3.525200,No log
6,2.882400,No log
7,2.263700,No log
8,1.697000,No log
9,1.666100,No log
10,1.395200,No log


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=1600, training_loss=1.294702771604061, metrics={'train_runtime': 7086.8155, 'train_samples_per_second': 1.785, 'train_steps_per_second': 0.226, 'total_flos': 3.85594947612672e+16, 'train_loss': 1.294702771604061, 'epoch': 50.0})

In [ ]:
# Save the model
trainer.model.save_pretrained("ESE577_AI_chatbot")
model.config.use_cache = True
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Line

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INST]@AI.Supervised.Unsupervised. {question}. [/INST]"
  return prompt

while True:
  question = input("Enter your AI-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()

Enter your AI-related question (hit Enter to exit): 


In [ ]:
# Mapping of answer choices to labels
answer_mapping = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
}

device = torch.device("cuda")

# Define function to validate model performance on multiple choice questions
def validate_model(file_path):
    correct_predictions = 0
    total_questions = 0

    with open(file_path, "r", encoding='utf-8') as f:
        lines = f.readlines()

        for line in lines:
            # Split question and correct answer
            question_answer = line.strip().split('</s>')
            if len(question_answer) != 2:
                continue

            question = question_answer[0] + "</s>"
            correct_answer = question_answer[1].strip()

            # Tokenize the question
            inputs = tokenizer(question, return_tensors="pt").to(device)

            # Generate the model's prediction
            outputs = model.generate(inputs['input_ids'], max_length=512)
            decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

            print(f"Question: {question}")
            print(f"Predicted Answer: {decoded_output}")
            print(f"Correct Answer: {correct_answer}")
            print()

# Path to your validation file
file_path = 'Validation_Data.txt'
validate_model(file_path)

Question: <s>[INST]@AI_Supervised_Unsupervised.What is the primary goal of clustering in unsupervised learning?.[/INST]A. To predict a categorical outcome variable. B. To group similar data points together. C. To reduce the dimensionality of the data. D. To find the optimal hyperplane separating different classes.</s>
Predicted Answer: [INST]@AI_Supervised_Unsupervised.What is the primary goal of clustering in unsupervised learning?.[/INST]A. To predict a categorical outcome variable. B. To group similar data points together. C. To reduce the dimensionality of the data. D. To find the optimal hyperplane separating different classes.

[B]@AI_Supervised_Supervised. What is the primary goal of regression in supervised learning?.[/

[C] Mapping input data to continuous output values.
Correct Answer: B

Question: <s>[INST]@AI_Supervised_Unsupervised.Which of the following best describes a "cluster" in the context of unsupervised learning?.[/INST]A. A single, isolated data point. B. A group 